In [198]:
import re
import pickle
import Stemmer
import numpy as np
from collections import defaultdict, OrderedDict
import sys

In [199]:
re.split(':','b:Sachin c:world cup')

['b', 'Sachin c', 'world cup']

In [200]:
STOP_DICT = {}
# INPUT_FILE = '../enwiki-20200801-pages-articles-multistream1.xml-p1p30303'
INDEX_DIR = 'index/'
QUERY = 'World cup'
if INDEX_DIR[-1]!='/':
    INDEX_DIR+='/'
STOP_DICT = {}
STOP_FILE = ''
if INDEX_DIR.split('/')[0] == '2018114017':
    STOP_FILE = '2018114017/stopwords.pickle'
else:
    STOP_FILE = 'stopwords.pickle'
with open(STOP_FILE, 'rb') as handle:
    STOP_DICT = pickle.load(handle)
handle.close()
stemmer = Stemmer.Stemmer('english')
f = open(INDEX_DIR+"index.txt", "r")
index_string = f.read()
f.close()
temp = index_string.split('\n')
index = defaultdict(list)
for i in temp:
    splits = i.split(':')
    index[splits[0]] = splits[1].split(' ')

In [201]:
def preprocess(text):
    text = text.lower()
    tokens = re.sub(r'[^A-Za-z0-9]+', r' ', text).split()
    stemmed_stop_free = []
    for token in tokens:
        if token not in STOP_DICT:
            stemmed_stop_free.append(stemmer.stemWord(token))
    return stemmed_stop_free

In [202]:
def parse_query(query):
    query_list = query.split(':')
    if len(query_list) == 1:
        return preprocess(query_list[0]) , 0
    else:
        query_dict = {}
        # query_dict[query_list[0]] = ''
        for i in range(1, len(query_list)-1):
            query_dict[query_list[i-1][-1]] = preprocess(query_list[i][:-2])
        query_dict[query_list[-2][-1]] = preprocess(query_list[-1])
        return query_dict, 1

In [203]:
def run_whole_query(query):
    docs_intersect = []
    docs_postlist = {}
    for i in range(len(query)):
        doc_list = []
        post = []
        for j in index[query[i]]:
            splits = re.split('d|b|i|l|r|t|c', j)
            doc_list.append(splits[1])
            post.append(j)
        docs_postlist[query[i]]=post
        if i == 0:
            docs_intersect = doc_list
        else:
            docs_intersect = np.intersect1d(docs_intersect, doc_list)
    return docs_intersect, docs_postlist

In [204]:
def run_parsed_query(query_dict):
    flag = 0
    docs_intersect = []
    docs_postlist = {}
    for category in query_dict.keys():
        for i in range(len(query_dict[category])):
            doc_list = []
            post = []
            for j in index[query_dict[category][i]]:
                if category == 't':
                    splits = re.split('t', j)
                if category == 'b':
                    splits = re.split('b', j)
                if category == 'i':
                    splits = re.split('i', j)
                if category == 'c':
                    splits = re.split('c', j)
                if category == 'l':
                    splits = re.split('l', j)
                if category == 'r':
                    splits = re.split('r', j)
                if len(splits) > 1:
                    post.append(j)
                    splits2 = re.split('d|b|i|l|r|t|c', j)
                    doc_list.append(splits2[1])
                    docs_postlist[category+':'+query_dict[category][i]]=j
            docs_postlist[category+':'+query_dict[category][i]]=post
            if flag == 0:
                docs_intersect = doc_list
                flag = 1
            else:
                docs_intersect = np.intersect1d(docs_intersect, doc_list)
    return docs_intersect, docs_postlist

In [205]:
def print_postlist(postlist, docs_ids):
    for i in postlist.keys():
        print("Postlist for",i+":")
        for j in postlist[i]:
            print(j,end=' ')
        print()
        print()

In [206]:
parsed, querytype = parse_query(QUERY)
docs_intersect = []
docs_postlist = {}
if querytype == 0:
    docs_intersect, docs_postlist = run_whole_query(parsed)
else:
    docs_intersect, docs_postlist = run_parsed_query(parsed)
print_postlist(docs_postlist, docs_intersect)

Postlist for world:
d1b7 d11b1 d17r1 d65b14r1 d67b5 d69b5 d70b5 d71b3 d72b5 d73b7 d75r1 d78b3 d80b4 d81r1 d82b2 d85b21i13l1r5 d86b5 d88b19r2 d89b1l1 d90b10 d92b1 d94b3 d97b13 d98b12r1 d99b21i1c1r1 d101b1i1r1 d103b4i1 d104b13i3l2r3 d110b7r1 d111b7 d113b15 d115b11l1r4 d116b23i1l2r1 d119b1 d120r1 d133b3 d134b1 d135b13l1 d138b6 d139b1r1 d147b2 d149b2 d150b3 d152b1 d154b1 d155b5l3 d156b3 d159b43i1r6 d160b7i2r2 d161r1 d165b19i2l1 d166b82i1r10 d167b22l1r5 d168b11r4 d169b3 d170b5i18c1r5 d171b2 d172b2i1l1 d173b4 d175b4 d176b17 d183b16c1l2r7 d184b33i3r4 d185b39i3r7 d186b3 d189b52i8l1r4 d190b2r1 d192b6 d193b13r1 d199b9l1r1 d200l1 d202b8r6 d204b1 d205b1 d208r1 d209b9r3 d210b7 d211b3 d212b1 d215b2 d216b1 d221b2 d223b6 d224b21i1r1 d226b4 d227b1 d237b2 d242b5r7 d243b9i23r1 d244b2 d245b48r5 d246b1 d247b31r3 d248b32 d250b6i1r2 d251b8i1r1 d253b20r3 d258b6l1 d259b3 d260b2 d261b2r1 d262b10r1 d264b6l1 d267b1r1 d268r3 d269b6i1r1 d273b1 d274b24r2 d275b13 d277b1i1 d278b8r2 d279b1i1 d280b8i9l1 d281b11i13l1 d28

In [207]:
# docs_postlist, parsed